In [1]:
import difflib
from fuzzywuzzy import fuzz
import itertools
import pandas as pd
from collections import Counter
import numpy as np
import string
import funcy
import re
import os
import uuid
import math

%load_ext autoreload

%autoreload 2

from data_cleaning_functions import (trans_remov_punc, standardize_whitespace, remove_punc, remove_suffix_from_last_name,
                                     clean_names, has_award, has_suffix, get_suffix, replace_last_name, 
                                     is_year_range, str_sim, clean_med_school, clean_std_college_name)

from dev import (
    APP_DATA_DIR, SUM_STAT_DIR, CARD_DATA_DIR, CORRECTIONS_DIR, AWARDS_KEYWORDS, NAME_COLS, RAW_NAME_COLS, 
    RAW_CARD_ID, RAW_INDEX_IDS, PERSON_APPLICATION_ID, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES,
    PICKLE_DIR)

from merging_functions import *

OUTPUT_CSV = True 
RAW_APPLICANT_DATA_FILENAME = 'raw_applicant_card_data.csv'
MISSING_APPDATE_FILENAME = 'index_cards_no_application_date.csv'
APP_SPELLING_CORRECTIONS = 'index_card_manual_corrections.xlsx'


PERSONAL_INFO = [
    'clean_first_name', 'clean_last_name', 'clean_middle_name',
    'date_of_birth', 'medical_school', 'clean_college_trans']


In [2]:
all_appcards2 = pd.read_csv(os.path.join(CARD_DATA_DIR, RAW_APPLICANT_DATA_FILENAME))

# drop all rows with first, last name NA
all_app3 = all_appcards2.dropna(subset=['application_date'], how='all')


def id_poorlyfmtdates(str_date):
    try:
        dt = pd.to_datetime(str_date, format='%m/%d/%Y')
        return True
    except (ValueError, AssertionError):
        return False

mask = all_app3.application_date.apply(id_poorlyfmtdates)
# all_app3.loc[~mask, :]

# change '3/31971 to 3/3/1971
# change 41/8/1966 to 4/8/1966 
all_app3.loc[all_app3.application_date=='3/31971', 'application_date'] = '3/3/1971'
all_app3.loc[all_app3.application_date=='41/8/1966', 'application_date'] = '4/8/1966'

# convert application date to date object
all_app3.loc[:, 'application_date'] = all_app3['application_date'].apply(lambda x: pd.to_datetime(x))

# do the same date check for birth date columns
bdate_mask = all_app3.date_of_birth.apply(id_poorlyfmtdates)
all_app3.loc[~bdate_mask, ['first_name', 'last_name', 'date_of_birth']]
all_app3.loc[(
        all_app3.last_name=='Cook') & 
                  (all_app3.middle_name=='James') & 
                  (all_app3.first_name.isnull()), 'date_of_birth'] = '1/27/1940'

all_app3[NAME_COLS] = all_app3[RAW_NAME_COLS].applymap(clean_names)

/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/pandas/core/frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [3]:
all_app3.loc[(all_app3.clean_last_name=='RUBENSTEIN') & (all_app3.clean_middle_name=='ALLAN'), 'application_date'] = pd.to_datetime('1/21/1973')

all_app3.loc[(
        all_app3.clean_last_name=='ROOT') & 
                  (all_app3.clean_first_name=='RICHARD'), 'application_date'] = pd.to_datetime('5/23/1963')

all_app3.loc[(
        all_app3.clean_last_name=='FREIDMAN') & (all_app3.clean_first_name=='STANFORD'), 'application_date'] = pd.to_datetime('01/01/1960')


In [4]:
all_app3.rename(columns={'medical_school':'original_medical_school'}, inplace=True)

/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [5]:
# deal with suffixes and honors
suffix_mask = all_app3.clean_last_name.apply(has_suffix)
# all_app_short.loc[suffix_mask, ['clean_last_name', 'clean_first_name', 'clean_middle_name']]

# for those last names that seem to have a suffix, pull into seperate column and keep everything last word of last name
all_app3.loc[suffix_mask, 'clean_suffix'] = all_app3.loc[suffix_mask, 'clean_last_name'].apply(get_suffix)
all_app3.loc[suffix_mask, 'clean_last_name'] = all_app3.loc[suffix_mask, 'clean_last_name'].apply(remove_suffix_from_last_name)

# some first names also contain some honors such as 'Pfizer Award' or 'Honor Society'
# these should be pulled into the honors and awards columns
has_award_fnc = funcy.rpartial(has_award, AWARDS_KEYWORDS)


# get a list of all the med school honors columns
honors_columns = [c for c in all_app3.columns if 'honor' in c]
print honors_columns

has_award_mask = all_app3['clean_first_name'].apply(has_award_fnc)

all_app3.loc[has_award_mask, 'extra_honor'] = all_app3.loc[has_award_mask, 'clean_first_name']

# create column mask for each row where one of the honors columns is blank
for hc in honors_columns:
    hc_mask = (has_award_mask) & (pd.isnull(all_app3[hc]))
    all_app3.loc[hc_mask, hc] = all_app3.loc[hc_mask, 'extra_honor']
# check for any columns that already have full honors and cant be filled
all_app3.loc[hc_mask, honors_columns].dropna(how='any')

# drop extra honor columns
all_app4 = all_app3.drop('extra_honor', axis=1)

# replace those honors first names with np.nan
all_app4.loc[has_award_mask, 'clean_first_name'] = np.nan

# for those columns where med school is equal to last name or med_school is a year range, delete
med_school_str_sim = funcy.rpartial(str_sim, 'medical_school', 'clean_last_name')
all_app4.loc[:, 'school_name_sim'] = all_app4.loc[:, ['clean_last_name', 'medical_school']].apply(med_school_str_sim, axis=1)

all_app4.loc[all_app4.school_name_sim > .6, :]

all_app4.loc[:, 'clean_college'] = all_app4.undergraduate_school.apply(clean_names)

to_remove_college = [
    ' AND ', ' AT ', 'THE ', ' COLLGE', 'UNIVERISTY', 'UNIVERWSITY', 'MASSACHUSSETTS', 'JOHN ', 'DE PAUW', 'ASBURY', 
'DREXEL INSTITUTE OF TECHNOLOGY', 'A B BROWN UNIVERSITY', 'DARTMOUTH MEDICAL SCHOOL', 'RENSSELAER UNIVERSITY', 
'RENSSELAER POLYTECHNICAL INSTITUTE', ' STE', 'COLLEGE OF HOLY CROSS', 'HOLLY CROSS', 'JOHNSS ',  'BERKLEY',
'UC ', 'PITTSBURRGH', 'WESLYN', 'WILLAMS', 'GEORGIA TECH', 'NEW YORK UNIVERSITY UNIV', 
'UNIVERSITY OF MICHIGAN IS A', 'OHIO', 'STATE UNIVERSITY OF NEW YORK AT BUFFALO']
to_replace_college = [
    ' ', ' ', ' ', ' COLLEGE', 'UNIVERSITY', 'UNIVERSITY', 'MASSACHUSETTS', 'JOHNS ', 'DEPAUW', 'ASHBURY',
    'DREXEL UNIVERSITY', 'BROWN UNIVERSITY', 'DARTMOUTH', 'RENSSELAER POLYTECHNIC INSTITUTE', 
    'RENSSELAER POLYTECHNIC INSTITUTE', ' STATE', 'HOLY CROSS', 'HOLY CROSS', 'JOHNS ', 
    ' BERKELEY', 'UNIVERSITY OF CALIFORNIA ', 'PITTSBURGH', 'WESLEYAN', 'WILLIAMS', 
    'GEORGIA INSTITUTE OF TECHNOLOGY', 'NEW YORK', 'UNIVERSITY OF MICHIGAN', 'OHIO STATE', 'SUNY BUFFALO']

clean_college_fnc = funcy.rpartial(clean_std_college_name, to_remove_college, to_replace_college)

# make college mispelling and different reference translation table
all_app4.loc[:, 'clean_college_trans'] = all_app4.clean_college.apply(clean_college_fnc)

all_app4.drop(['clean_college', 'school_name_sim'], axis=1, inplace=True)

/home/lraymond/MIT/Azoulay_2016/yellow_berets/yb/local/lib/python2.7/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


['honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third']


In [6]:
all_app4.loc[:, 'medical_school'] = all_app4.original_medical_school.apply(clean_med_school)

In [7]:
# set 'other' values to null
all_app4.loc[all_app4.medical_school=='OTHER', 'medical_school'] = np.nan

In [8]:
# need to convert undergrad_year_grad and med_school_grad to numbers to maintain consistency
all_app4.loc[:, ['undergrad_year_grad', 'medschool_year_grad']] = all_app4.loc[:, ['undergrad_year_grad', 'medschool_year_grad']].apply(
    lambda x: pd.to_numeric(x, errors='coerce'))

# now, sort by names, med school, undergrad school, 
all_app5 = all_app4.sort_values(by=PERSONAL_INFO)

# LAST_NAME_MISSPELLINGS = {
#         'HOMCY': 'HOMEY', 'DROBIS': 'DROBIN', 'DEFRONZO':'DEFRENZO', 
#         'BRADEN 3R': 'BRADEN', 'BORKER': 'BORER', 'CASTLES': 'CASTLE',
#         'CUONO': 'CUOMO', 'CYRULNIK': 'CYRULINK', 'EISENBATH': 'EISENBARTH', 
#         'ELLIOTT': 'ELIOT', 'FINKLESTEIN': 'FINKELSTEIN', 'HEINRICK': 'HEINRICH', 
#         'HERLIKY': 'HERLIHY', 'HIMMELHOCK': 'HIMMELHOCH', 'JANOWSKY': 'JANKOWSKY', 
#         'KLINENBERG': 'KLINEBERG', 'KORNFELD': 'KORNFIELD', 'NEIDORF': 'NEIDOFT',
#         'OLEINICK': 'OLENICK', 'ROSKES': 'ROSKE'
# }
# removed CUONO, DROBIS, 


LAST_NAME_MISSPELLINGS = {
        'HOMCY': 'HOMEY', 
        'BRADEN 3R': 'BRADEN', 'BORKER': 'BORER', 'CASTLES': 'CASTLE',
        'CYRULNIK': 'CYRULINK', 'EISENBATH': 'EISENBARTH', 
        'HEINRICK': 'HEINRICH', 
        'HERLIKY': 'HERLIHY', 'HIMMELHOCK': 'HIMMELHOCH', 'JANOWSKY': 'JANKOWSKY', 
        'KLINENBERG': 'KLINEBERG', 'KORNFELD': 'KORNFIELD', 'NEIDORF': 'NEIDOFT',
        'OLEINICK': 'OLENICK', 'ROSKES': 'ROSKE'
}

replace_last_name_fnc = funcy.rpartial(replace_last_name, LAST_NAME_MISSPELLINGS)

# correct last name mispellings
all_app5.loc[:, 'clean_last_name'] = all_app5.loc[:, 'clean_last_name'].apply(replace_last_name_fnc)
all_app5.loc[all_app5.clean_last_name=='MORTON', 'clean_first_name'] = 'JOHN'

# convert ca column to float62
all_app5.loc[:, 'ca'] = all_app5.loc[:, 'ca'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [9]:
female_mask = (
    (all_app5['clean_first_name'].isin(FEMALE_FIRST_NAMES)))
#     | (
#         all_app5['clean_middle_name'].isin(FEMALE_MIDDLE_NAMES)))
all_app5['is_female'] = 0
all_app5.loc[female_mask & ~pd.isnull(all_app5['clean_first_name']), 'is_female'] = 1

In [10]:
def get_first_letter(str_var):
    if pd.isnull(str_var) or str_var=='':
        return np.nan
    return str_var[0]

In [11]:
last_names = ['RIPLEY', 'HAYWOOD', 'HAYWARD', 'PERPICH', 'BRADEN', 'BRADEN R', 'BULKEY', 'CHESEBRE', 'DIEZMAN']

all_app5.loc[all_app5.clean_last_name.isin(last_names), NAME_COLS +['medical_school', 'medschool_grad_year']]

,clean_first_name,clean_middle_name,clean_last_name,medical_school,medschool_grad_year
4309,BRUCE,WILCOX,CHESEBRE,HARVARD MEDICAL SCHOOL,NaN
4502,DALE,E,DIEZMAN,NaN,NaN
4076,GREGORY,BARTLETT,BULKEY,HARVARD MEDICAL SCHOOL,NaN
5156,HUGH,R,HAYWOOD,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE,NaN
6424,JOSEPH,GEORGE,PERPICH,UNIVERSITY OF MINNESOTA MEDICAL SCHOOL - DULUTH,NaN
4041,WILLIAM,NaN,BRADEN R,HARVARD MEDICAL SCHOOL,NaN
342,NaN,NaN,BRADEN,HARVARD MEDICAL SCHOOL,NaN
1500,NaN,NaN,HAYWARD,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE,NaN
2850,NaN,GEORGE,PERPICH,UNIVERSITY OF MINNESOTA MEDICAL SCHOOL - TWIN ...,NaN
3224,NaN,C,RIPLEY,GEORGE WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,NaN


In [12]:
# merge in manual corrections excel sheet
manual_fixes = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'index_card_manual_corrections.xlsx')).rename(
    columns={
        'clean_medical_school': 'medical_school', 
        'to_fix_clean_medical_school': 'to_fix_medical_school'}).drop('Unnamed: 9', axis=1)

In [13]:
c = filter(lambda x: '_year' not in x, manual_fixes.columns)
manual_fixes.loc[:, c] = manual_fixes[c].applymap(clean_names)

In [14]:
manual_fixes.loc[manual_fixes.medical_school=='CHICAGO', 'medical_school'] = 'UNIVERSITY OF CHICAGO'

In [15]:
manual_fixes.loc[manual_fixes.clean_first_name==' ', 'clean_first_name'] = np.nan
manual_fixes.loc[manual_fixes.clean_middle_name==' ', 'clean_middle_name'] = np.nan

In [16]:
missing_names = pd.merge(
    left=all_app5, right=manual_fixes, on=[
        'clean_first_name', 'clean_middle_name', 'clean_last_name'], how='inner', suffixes=['_1', '_2'])
missing_names['med_sim'] = missing_names[['medical_school_1', 'medical_school_2']].apply(
    lambda x: get_name_str_sim(x), axis=1)

In [17]:
missing_names1 = missing_names.sort_values(
    'med_sim', ascending=False).drop_duplicates(NAME_COLS).drop(['medical_school_2', 'med_sim'], axis=1)
missing_names1 = missing_names1.rename(columns={'medical_school_1': 'medical_school'})

In [18]:
missing_names.loc[missing_names.clean_last_name=='BULKEY']

,Unnamed: 0,raw_uuid,address,age,application_date,associate_program_entered,bob,ca,cc,cord,...,clean_college_trans,medical_school_1,is_female,medical_school_2,to_fix_clean_first_name,to_fix_clean_middle_name,to_fix_clean_last_name,to_fix_medical_school,to_fix_medschool_year_grad,med_sim


In [19]:
# for the people who match, consolidate the columns

# missing_names = pd.merge(
#     left=all_app5, right=manual_fixes, on=['clean_first_name', 'clean_middle_name', 'clean_last_name', 'medical_school'], how='inner')

print manual_fixes.shape
print missing_names1.shape

missing_names1A = missing_names1.rename(columns={
            'clean_first_name': 'old_first',
            'clean_middle_name': 'old_middle',
            'clean_last_name': 'old_last'}) 
missing_names1B = missing_names1A.rename(columns={
            'to_fix_clean_first_name': 'clean_first_name', 
            'to_fix_clean_middle_name': 'clean_middle_name', 
            'to_fix_clean_last_name': 'clean_last_name'}) 

(632, 9)
(541, 78)


In [20]:
missing_names1B.loc[missing_names1B.clean_last_name.isin(['ELLER', 'EILER']), NAME_COLS]
# missing_names.loc[missing_names.clean_last_name.isin(['ELLER', 'EILER']), NAME_COLS+['to_fix_clean_last_name']]

,clean_first_name,clean_middle_name,clean_last_name
15,DONALD,MARTIN,EILER
282,DONALD,MARTIN,EILER


In [21]:
# drop extra columns
missing_names2 = missing_names1B.drop(
    (c for c in missing_names1B.columns if c not in all_app5.columns), axis=1).drop_duplicates([RAW_CARD_ID])

all_app6  = pd.concat(
    [all_app5.loc[~all_app5[RAW_CARD_ID].isin(missing_names2[RAW_CARD_ID]), :], missing_names2], axis=0)

In [22]:
missing_names2.loc[missing_names2.clean_last_name=='EILER']

,Unnamed: 0,raw_uuid,address,age,application_date,associate_program_entered,bob,ca,cc,cord,...,reviewer,undergrad_year_grad,application_year,clean_suffix,clean_college_trans,medical_school,is_female,clean_first_name,clean_middle_name,clean_last_name
15,4597,4597,506 W. Edgar,NaN,1964-10-28,CA,0.0,1.0,1.0,1.0,...,2,NaN,1964.0,NaN,NaN,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,0,DONALD,MARTIN,EILER
282,922,922,506 West Edgar,NaN,1964-10-28,NaN,0.0,1.0,1.0,1.0,...,1,NaN,1964.0,NaN,NaN,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,0,DONALD,MARTIN,EILER


In [23]:
# all_app5.loc[(
#         all_app5.clean_last_name.isin(mf.clean_last_name)) & (
#         all_app5.clean_middle_name.isin(mf.clean_middle_name)), NAME_COLS+['medical_school']].to_csv('test.csv')
all_app5.loc[(
        all_app5.clean_last_name == 'HAYWARD'), NAME_COLS+['medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,medical_school
1500,NaN,NaN,HAYWARD,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE


In [24]:
mf = manual_fixes.loc[~manual_fixes.to_fix_clean_last_name.isin(missing_names2.clean_last_name), NAME_COLS+['medical_school']]

In [25]:
print mf.shape

(0, 4)


In [26]:
last_name = 'ELLER'

# f_name = 'ADAM'
# m_name = 'NORMAN'
# all_app5.loc[(
#         all_app5.clean_first_name==f_name) & (all_app5.clean_middle_name==m_name), NAME_COLS+['medical_school', 'residency_hospital', 'internship_hospital_1']]
all_app5.loc[all_app5.clean_last_name==last_name, NAME_COLS+['medical_school', 'residency_hospital', 'internship_hospital_1']]
all_app6.loc[all_app6.clean_last_name==last_name, NAME_COLS+['medical_school', 'residency_hospital', 'internship_hospital_1']]
# missing_names.loc[missing_names.clean_last_name==last_name, NAME_COLS+['medical_school']]
# manual_fixes.loc[manual_fixes.clean_first_name=='LAWRENCE',  NAME_COLS+['medical_school']]
# manual_fixes.loc[manual_fixes.clean_last_name==last_name,  NAME_COLS+['medical_school']]
# all_app6.loc[all_app6.clean_last_name==last_name,  NAME_COLS+['medical_school', 'medschool_year_grad']]

,clean_first_name,clean_middle_name,clean_last_name,medical_school,residency_hospital,internship_hospital_1


In [27]:
all_app6['clean_first_initial'] = all_app6.clean_first_name.apply(get_first_letter)
all_app6['clean_middle_initial'] = all_app6.clean_middle_name.apply(get_first_letter)
all_app6['application_year'] = all_app6.application_date.apply(lambda x: pd.to_datetime(x).year)


In [28]:
all_app6.loc[(all_app6['application_year'] > 1990) | (all_app6['application_year'] < 1950), 'application_year'] = np.nan

In [29]:
# drop people with female names and two columns
is_female_mask = (all_app6.clean_first_name.isin(FEMALE_FIRST_NAMES))
all_app6['is_female'] = 0
all_app6.loc[is_female_mask, 'is_female'] = 1
# RENAME INTERNSHIP HOSPITAL COL
all_app7 = all_app6.drop(['Unnamed: 0',"daniel's_comments"], axis=1).rename(
    columns={'internship_hospital_1': 'internship_hospital'})

In [30]:
# check for any remaining females, check to make sure fuzzy matches worked properly
all_app7.loc[all_app7.clean_last_name=='EILER', NAME_COLS]

,clean_first_name,clean_middle_name,clean_last_name
15,DONALD,MARTIN,EILER
282,DONALD,MARTIN,EILER


In [31]:
all_app7.loc[all_app7.clean_last_name=='LEVINE', NAME_COLS+['reviewer']]

,clean_first_name,clean_middle_name,clean_last_name,reviewer
5953,ALAN,MARC,LEVINE,2
5815,ARTHUR,SAMUEL,LEVINE,2
5838,DAVID,NORMAN,LEVINE,2
5851,FREDERICK,HUGH,LEVINE,2
5861,JOEL,B,LEVINE,2
5871,MARSHALL,ANDREW,LEVINE,2
5872,ROBERT,AARON,LEVINE,2
5753,ROBERT,J,LEVINE,2
5933,RODNEY,LEE,LEVINE,2
2240,NaN,ANDREW,LEVINE,1


In [32]:
# split into reviewer 1 and 2, and try to match
all_app7['fuzzy_merge_col'] = all_app7[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
rev2 = df_get_closest_matches(all_app7[all_app7.reviewer==2], all_app7[all_app7.reviewer==1], 'fuzzy_merge_col') 


feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'clean_last_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
    'application_year': get_dt_sim,
    'address': get_name_str_sim
}

rev3 = add_similarity_features(rev2, feature_dict, check_match)

rev1_counter = Counter(all_app7[all_app7.reviewer==1].clean_last_name.values)
rev2_counter = Counter(all_app7[all_app7.reviewer==2].clean_last_name.values)
rev3['last_name_counts_1'] = rev3.clean_last_name_1.apply(lambda x: rev1_counter[x])
rev3['last_name_counts_2'] = rev3.clean_last_name_2.apply(lambda x: rev2_counter[x])

# now, sort by is_match, similarity scores and only keep 1 uuid from each data set
last_name_unique_mask = (
    (rev3.last_name_counts_1==1) & (rev3.last_name_counts_2==1) & (
        rev3.application_year_sim<4) & (rev3.medical_school_sim > .8))
rev3.loc[last_name_unique_mask, 'is_match'] = 1

In [33]:
rev3.is_match.describe()

count    3759.000000
mean        0.795956
std         0.403055
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: is_match, dtype: float64

In [34]:
sims_cols = ['medical_school_sim', 'address_sim', 'clean_middle_name_sim', 'clean_first_name_sim']

rev4 = rev3.loc[~pd.isnull(rev3.index), :].sort_values([
        'raw_uuid_2', 'raw_uuid_1', 'is_match']+sims_cols, ascending=False)
print rev3.shape
print rev4.shape
rev5 = filter_one_match_per_group(rev4, 'raw_uuid_1', {'raw_uuid_2': 'raw_uuid_3'}, sims_cols)
print rev5.shape
rev6 = filter_one_match_per_group(rev5, 'raw_uuid_2', {'raw_uuid_1': 'raw_uuid_4'}, sims_cols)
print rev6.shape

(3759, 155)
(3636, 155)
                      raw_uuid_1  raw_uuid_3
JAMES L SCHERER           7084.0        3493
JOHN STONER               7078.0        3487
JOHN PATRICK O GRADY      6406.0        2790
STEPHEN KARAS             5617.0        1971
OLIVER WILLIAM JONES      5449.0        1797
PAUL HINKES               5308.0        1656
MARC A FRADER             4801.0        1134
(2965, 157)
Empty DataFrame
Columns: [raw_uuid_2, raw_uuid_4]
Index: []
(2961, 159)


In [35]:
# del rev6['raw_uuid_4']
rev6.loc[rev6.clean_last_name_1=='LEVINE', 'clean_first_name_1']

1119        ALAN
1138      RODNEY
1192      ROBERT
1193    MARSHALL
1204        JOEL
1220       DAVID
1238      ARTHUR
Name: clean_first_name_1, dtype: object

In [36]:
matched_ids = np.concatenate([
        rev6.raw_uuid_1.dropna().unique(), rev6.raw_uuid_2.dropna().unique(), rev6.raw_uuid_3.dropna().unique()], 
                        axis=0)
print matched_ids.shape

(5929,)


In [37]:
# try to merge on middle initial and clean last name

unmatched_r1 = get_nonmatched(all_app7[all_app7.reviewer==1], matched_ids, 'raw_uuid')
unmatched_r2 = get_nonmatched(all_app7[all_app7.reviewer==2], matched_ids, 'raw_uuid')

unmatched_r1['fuzzy_merge_col'] = unmatched_r1[
    ['clean_middle_initial', 'clean_last_name']].apply(create_str_merge, axis=1)
unmatched_r2['fuzzy_merge_col'] = unmatched_r2[
    ['clean_middle_initial', 'clean_last_name']].apply(create_str_merge, axis=1)

(743, 74)
(622, 74)


In [38]:
rev2_counter = Counter(unmatched_r2.clean_last_name.values)
rev1_counter = Counter(unmatched_r1.clean_last_name.values)

match_round2 = df_get_closest_matches(unmatched_r2, unmatched_r1, 'fuzzy_merge_col')

In [39]:
middle_last3 = add_similarity_features(match_round2, feature_dict, check_match)

middle_last3['last_name_counts_1'] = middle_last3.clean_last_name_1.apply(lambda x: rev1_counter[x])
middle_last3['last_name_counts_2'] = middle_last3.clean_last_name_2.apply(lambda x: rev2_counter[x])

In [40]:
# now, sort by is_match, similarity scores and only keep 1 uuid from each data set
last_name_unique_mask = (
    (middle_last3.last_name_counts_1==1) & (middle_last3.last_name_counts_2==1) & (
        middle_last3.application_year_sim<2) & (middle_last3.medical_school_sim > .8))
middle_last3.loc[last_name_unique_mask, 'is_match'] = 1

In [41]:
middle_last4 = middle_last3.loc[~pd.isnull(middle_last3.index), :].sort_values([
        'raw_uuid_2', 'raw_uuid_1', 'is_match']+sims_cols, ascending=False)
print middle_last3.shape
print middle_last4.shape
middle_last5 = filter_one_match_per_group(middle_last4, 'raw_uuid_1', {'raw_uuid_2': 'raw_uuid_3'}, sims_cols)
print middle_last5.shape
middle_last6 = filter_one_match_per_group(middle_last5, 'raw_uuid_2', {'raw_uuid_1': 'raw_uuid_4'}, sims_cols)
print middle_last6.shape

(847, 155)
(824, 155)
Empty DataFrame
Columns: [raw_uuid_1, raw_uuid_3]
Index: []
(536, 157)
Empty DataFrame
Columns: [raw_uuid_2, raw_uuid_4]
Index: []
(536, 159)


In [42]:
middle_last6.loc[~pd.isnull(middle_last6.raw_uuid_4),['raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3']]

,raw_uuid_1,raw_uuid_2,raw_uuid_3


In [43]:
middle_last6.loc[middle_last6.clean_last_name_1=='LEVINE', sims_cols +[
 'clean_first_name_1', 'clean_middle_name_1','clean_middle_name_2', 'clean_first_name_2']]
# matches.loc[matches.clean_last_name=='LEVINE', 'clean_first_name']

,medical_school_sim,address_sim,clean_middle_name_sim,clean_first_name_sim,clean_first_name_1,clean_middle_name_1,clean_middle_name_2,clean_first_name_2
205,100.0,92.0,100.0,NaN,FREDERICK,HUGH,HUGH,NaN
225,100.0,86.0,100.0,NaN,ROBERT,J,J,NaN


In [44]:
matched_ids = np.concatenate([
        rev6.raw_uuid_1.dropna().unique(), rev6.raw_uuid_2.dropna().unique(), rev6.raw_uuid_3.dropna().unique(), 
        middle_last6.raw_uuid_1.dropna().unique(), middle_last6.raw_uuid_2.dropna().unique(), middle_last6.raw_uuid_3.dropna().unique()], 
                        axis=0)
print matched_ids.shape

(7001,)


In [45]:
# people who don't match on first or last
all_app7a = all_app7.drop('fuzzy_merge_col', axis=1)
unmatched = get_nonmatched(all_app7a, matched_ids, 'raw_uuid')

(777, 73)


In [46]:
matches = pd.concat([rev6, middle_last6], axis=0).drop(['raw_uuid_1_duplicate', 'raw_uuid_2_duplicate'], axis=1)
matches1 = consolidate_merge_cols(matches, ['_1', '_2'], ['application_year', 'raw_uuid'])

['address', 'age', 'application_date', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_name', 'clean_last_name', 'clean_middle_name', 'clean_suffix', 'clinical', 'cord', 'date_of_birth', 'dbs', 'fifth', 'first_name', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'internship_hospital', 'internship_year(s)', 'is_female', 'last_name', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', 'nhli', 'niaid', 'niamd', 'niamdd', 'nichd', 'nichhd', 'nidr', 'niehs', 'nigms', 'nimh', 'nindb', 'ninds', 'oir', 'original_medical_school', 'other', 'pharm_ra', 'pi', 'ra', 'rejected', 'rejection_date', 'research', 'residency_hospital', 'residency_type', 'residency_year(s)', 'reviewer', 'sa', 'sixth', 'state', 'teaching', 'undergrad_year_grad', 'undergraduate_school', 'withdrawal', 'year_accepted', 'zip_code', 'clean_first_initial', 'clean_middle_initial', 'last_

In [47]:
# append people by reviewer 3 and people not matched but reviewed by reviewer 1 or 2
full_matches = pd.concat([matches1, unmatched],
                      axis=0, ignore_index=True).sort_values(
                            ['clean_last_name', 'clean_middle_name', 'clean_first_name']).reset_index(drop=True)

In [48]:
full_matches1 = full_matches.drop([c for c in full_matches.columns if c.endswith('_sim') or '_counts' in c or c.endswith('_duplicate')], axis=1)

In [49]:
full_matches[~pd.isnull(full_matches.raw_uuid_4)]

,address,address_sim,age,application_date,application_year,application_year_1,application_year_2,application_year_sim,associate_program_entered,bob,...,reviewer,sa,sixth,state,teaching,undergrad_year_grad,undergraduate_school,withdrawal,year_accepted,zip_code


In [50]:
# remove duplicate data from application year
full_matches1.loc[pd.isnull(full_matches1.application_year), 'application_year'] = full_matches1.loc[
    pd.isnull(full_matches1.application_year), 'application_year_2'] 
full_matches1.loc[pd.isnull(full_matches1.application_year), 'application_year'] = full_matches1.loc[
    pd.isnull(full_matches1.application_year), 'application_year_1'] 

full_matches1.loc[pd.isnull(full_matches1.application_year_1), 'application_year_1'] = full_matches1.loc[
    pd.isnull(full_matches1.application_year_1), 'application_year_2'] 

full_matches1.loc[full_matches1.application_year_1==full_matches1.application_year_2, ]
full_matches1.loc[~pd.isnull(full_matches1.application_year_2), ]

dup_app_year_mask= full_matches1.application_year==full_matches1.application_year_2
full_matches1.loc[dup_app_year_mask, 'application_year_2'] = np.nan

dup_app_year_mask= full_matches1.application_year_1==full_matches1.application_year_2
full_matches1.loc[dup_app_year_mask, 'application_year_2'] = np.nan

dup_app_year_mask= full_matches1.application_year==full_matches1.application_year_1
full_matches1.loc[dup_app_year_mask, 'application_year_1'] = np.nan



full_matches1.loc[~pd.isnull(full_matches1.application_year_2), ['application_year', 'application_year_1', 'application_year_2']]

,application_year,application_year_1,application_year_2


In [51]:
# remove uuid dups

full_matches1.loc[pd.isnull(full_matches1.raw_uuid), 'raw_uuid'] = full_matches1.loc[
    pd.isnull(full_matches1.raw_uuid), 'raw_uuid_2'] 
full_matches1.loc[pd.isnull(full_matches1.raw_uuid), 'raw_uuid'] = full_matches1.loc[
    pd.isnull(full_matches1.raw_uuid), 'raw_uuid_1'] 

full_matches1.loc[pd.isnull(full_matches1.raw_uuid_1), 'raw_uuid_1'] = full_matches1.loc[
    pd.isnull(full_matches1.raw_uuid_1), 'raw_uuid_2'] 

dup_uuid_mask= full_matches1.raw_uuid==full_matches1.raw_uuid_2
full_matches1.loc[dup_uuid_mask, 'raw_uuid_2'] = np.nan

dup_uuid_mask= full_matches1.raw_uuid_1==full_matches1.raw_uuid_2
full_matches1.loc[dup_uuid_mask, 'raw_uuid_2'] = np.nan

dup_uuid_mask= full_matches1.raw_uuid==full_matches1.raw_uuid_1
full_matches1.loc[dup_uuid_mask, 'raw_uuid_1'] = np.nan

dup_uuid_mask= full_matches1.raw_uuid==full_matches1.raw_uuid_3
full_matches1.loc[dup_uuid_mask, 'raw_uuid_3'] = np.nan
# full_matches1.loc[full_matches1.raw_uuid_1==full_matches1.raw_uuid_2, ['raw_uuid_1', 'raw_uuid_2']]
full_matches1.loc[~pd.isnull(full_matches1.raw_uuid_3), ['raw_uuid', 'raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3']]

,raw_uuid,raw_uuid_1,raw_uuid_2,raw_uuid_3


In [52]:
full_matches2 = full_matches1.drop(['raw_uuid_2', 'raw_uuid_3', 'application_year_2', 'raw_uuid_4'], axis=1)

In [53]:
problem_names = ['PRZYBYLA', 'EILER', 'ELLER', 'BUCKLEY', 'BUKLEY', 'PERPICH', 'DIEZMAN', 'DIETZMAN', 'BRADEN R', 
                'GLASSBROTH', 'CHESEBRE', 'CHESEBRO', 'COLLIN', 'COLLINS', 'KETOVER', 'KENTOVER', 'BRADEN R', 
                'KNOWLER', 'SARAI', 'SARAL', 'COLDBERG', 'GOLDBERG']

In [54]:
full_matches2.loc[full_matches2.clean_last_name.isin(problem_names), NAME_COLS]

,clean_first_name,clean_middle_name,clean_last_name
495,MORTIMER,NaN,BUCKLEY
615,BRUCE,WILCOX,CHESEBRO
692,JAMES,E,COLDBERG
701,ROBERT,CHAPLIN,COLLINS
702,STEVEN,HAMES,COLLINS
867,DALE,E,DIETZMAN
952,DONALD,MARTIN,EILER
1341,MICHAEL,ELLIS,GOLDBERG
1342,ALAN,H,GOLDBERG
1343,NELSON,HOWARD,GOLDBERG


In [55]:
# del all_app3, all_appcards2, all_app4, all_app5, all_app6, all_app7a

In [56]:
full_matches2['fuzzy_merge_col'] = full_matches2[
    ['clean_middle_initial', 'clean_last_name', 'medical_school']].apply(create_str_merge, axis=1)

last_name_counter = Counter(full_matches2.clean_last_name)
full_matches2.loc[:, 'last_name_counts'] = full_matches2.apply(
    lambda x: last_name_counter[x['clean_last_name']], axis=1)

possible_dups = full_matches2[full_matches2['last_name_counts']>1].sort_values(
    ['clean_last_name', 'clean_middle_name', 'medical_school', 'city', 'application_year'])

people_match = df_get_closest_matches(possible_dups, possible_dups, 'fuzzy_merge_col', suffixes=['_x', '_y'])

In [57]:
# for people with the same information, drop from the data set
same_person_mask = (
        (people_match.raw_uuid_x==people_match.raw_uuid_y) & (people_match.raw_uuid_1_x==people_match.raw_uuid_1_y))

# need to add a second mask for people who have only 1 uuid
same_person_mask2 = (
        (people_match.raw_uuid_x==people_match.raw_uuid_y) & (pd.isnull(people_match.raw_uuid_1_x)) &
            (pd.isnull(people_match.raw_uuid_1_y)))

people_match2 = people_match[~(same_person_mask | same_person_mask2)]
print people_match2.shape

(298, 154)


In [58]:
# NOW, need to redo the merging process, but merge in based on same people, not just same application year
def check_similar(row):
        # address and application year match
    if row['application_year_sim'] > 3:
        return 0
    if row['medical_school_sim'] > 80 and row['clean_middle_name_sim'] > 60:
        return 1
    if row['medical_school_sim'] > 80 and row['clean_first_name_sim'] > 60:
        return 1
    if row['address_sim'] > 60 and row['medical_school_sim'] > 80 and (
            pd.isnull(row['clean_first_name_sim']) or row['clean_first_name_sim'] > 80) :
        return 1
    return 0



In [59]:
people_match3 = add_similarity_features(people_match2, feature_dict, check_similar, suffixes=['_x', '_y'])

merging_functions.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[sim_colname] = df[[col1, col2]].apply(fnc_val, axis=1)
merging_functions.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['is_match'] = 0


In [60]:
people_match3.loc[people_match3.clean_last_name_x.isin(['ALEXANDER', 'ALPERT']), 
                  ['is_match', 'clean_first_name_sim', 
                   'clean_middle_initial_x', 'clean_first_name_x','clean_first_name_y', 'medical_school_sim', 'clean_middle_name_x', 'clean_middle_name_y']]

,is_match,clean_first_name_sim,clean_middle_initial_x,clean_first_name_x,clean_first_name_y,medical_school_sim,clean_middle_name_x,clean_middle_name_y
C ALEXANDER DUKE MEDICAL SCHOOL,1,100.0,C,JOHN,JOHN,100.0,CHARLES,CHARLES
C ALEXANDER DUKE MEDICAL SCHOOL,1,100.0,C,JOHN,JOHN,100.0,CHARLES,CHARLES
S ALPERT HARVARD MEDICAL SCHOOL,1,100.0,S,JOSEPH,JOSEPH,100.0,STEPHEN,STEPHEN
S ALPERT HARVARD MEDICAL SCHOOL,1,100.0,S,JOSEPH,JOSEPH,100.0,STEPHEN,STEPHEN


In [61]:
people_match4 = people_match3[people_match3['is_match']==1].reset_index().drop_duplicates(subset='index',keep='first')
people_match4.shape

(119, 162)

In [62]:
people_match5 = consolidate_merge_cols(people_match4, ['_x', '_y'], ['application_year', 'application_year_1', 'raw_uuid', 'raw_uuid_1'])
people_match6 = people_match5.drop([c for c in full_matches.columns if c.endswith('_sim') or '_counts' in c or c.endswith('_duplicate')], axis=1)

['address', 'age', 'application_date', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_initial', 'clean_first_name', 'clean_last_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'clinical', 'cord', 'date_of_birth', 'dbs', 'fifth', 'first_name', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'internship_hospital', 'internship_year(s)', 'is_female', 'is_match', 'last_name', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', 'nhli', 'niaid', 'niamd', 'niamdd', 'nichd', 'nichhd', 'nidr', 'niehs', 'nigms', 'nimh', 'nindb', 'ninds', 'oir', 'original_medical_school', 'other', 'pharm_ra', 'pi', 'ra', 'rejected', 'rejection_date', 'research', 'residency_hospital', 'residency_type', 'residency_year(s)', 'reviewer', 'sa', 'sixth', 'state', 'teaching', 'undergrad_year_grad', 'undergraduate_school', 'withdrawal', 'year_accepted', 'zip_c

In [63]:
# consolidate uuid columns
people_match6['raw_uuid'] = people_match6['raw_uuid_x']
people_match6['raw_uuid_1'] = people_match6['raw_uuid_1_x']
people_match6['raw_uuid_2'] = np.nan
people_match6['raw_uuid_3'] = np.nan

fill_in_y = (
    (people_match6.raw_uuid_y!=people_match6.raw_uuid_x) & (people_match6.raw_uuid_y!=people_match6.raw_uuid_1))

people_match6.loc[fill_in_y, 'raw_uuid_2'] = people_match6.loc[fill_in_y, 'raw_uuid_y']

fill_in_y_1 = (
    (people_match6.raw_uuid_1_y!=people_match6.raw_uuid) & (people_match6.raw_uuid_1_y!=people_match6.raw_uuid_1) &
        (people_match6.raw_uuid_1_y!=people_match6.raw_uuid_2))

people_match6.loc[fill_in_y_1, 'raw_uuid_3'] = people_match6.loc[fill_in_y_1, 'raw_uuid_1_y']

people_match6.loc[pd.isnull(people_match6.raw_uuid_1), 'raw_uuid_1'] = people_match6.loc[pd.isnull(people_match6.raw_uuid_1), 'raw_uuid_3']
people_match6.loc[pd.isnull(people_match6.raw_uuid_1), 'raw_uuid_1'] = people_match6.loc[pd.isnull(people_match6.raw_uuid_1), 'raw_uuid_2']

people_match6.loc[people_match6.raw_uuid_1==people_match6.raw_uuid_3, 'raw_uuid_3'] = np.nan

# drop all uuidds except for i, 2, 2
UUID_COLS = ['raw_uuid', 'raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3']

people_match7 = people_match6.drop(['raw_uuid_x', 'raw_uuid_y', 'raw_uuid_1_x', 'raw_uuid_1_y'], axis=1)

In [64]:
# consolidate_app_year columns
# consolidate uuid columns
people_match7['application_year'] = people_match7['application_year_x']
people_match7['application_year_1'] = people_match7['application_year_1_x']
people_match7['application_year_2'] = np.nan
people_match7['application_year_3'] = np.nan

fill_in_y = (
    (people_match7.application_year_y!=people_match7.application_year_x) & (people_match7.application_year_y!=people_match7.application_year_1))

people_match7.loc[fill_in_y, 'application_year_2'] = people_match7.loc[fill_in_y, 'application_year_y']

fill_in_y_1 = (
    (people_match7.application_year_1_y!=people_match7.application_year) & (people_match7.application_year_1_y!=people_match7.application_year_1) &
        (people_match7.application_year_1_y!=people_match7.application_year_2))

people_match7.loc[fill_in_y_1, 'application_year_3'] = people_match7.loc[fill_in_y_1, 'application_year_1_y']

people_match7.loc[pd.isnull(people_match7.application_year_1), 'application_year_1'] = people_match7.loc[pd.isnull(people_match7.application_year_1), 'application_year_3']
people_match7.loc[pd.isnull(people_match7.application_year_1), 'application_year_1'] = people_match7.loc[pd.isnull(people_match7.application_year_1), 'application_year_2']

people_match7.loc[people_match7.application_year_1==people_match7.application_year_3, 'application_year_3'] = np.nan
people_match7.loc[people_match7.application_year_2==people_match7.application_year_3, 'application_year_3'] = np.nan
people_match7.loc[people_match7.application_year_1==people_match7.application_year_2, 'application_year_2'] = np.nan

# drop all uuidds except for i, 2, 2
APPLICATION_YEAR_COLS = ['application_year', 'application_year_1']

people_match8 = people_match7.drop(['application_year_2', 'application_year_3', 'application_year_x', 'application_year_y', 'application_year_1_x', 'application_year_1_y'], axis=1)

In [65]:
people_match4.loc[people_match6.clean_last_name.isin(['ALEXANDER', 'ALPERT']), [
        'application_year_x', 'application_year_1_x', 'application_year_y', 'application_year_1_y', 'clean_last_name_x']]

,application_year_x,application_year_1_x,application_year_y,application_year_1_y,clean_last_name_x
42,1971.0,NaN,1971.0,NaN,ALEXANDER
184,1969.0,NaN,1968.0,NaN,ALPERT


In [66]:
people_match8.loc[people_match8.clean_last_name.isin(['HERSH', 'ALPERT']), NAME_COLS]

,clean_first_name,clean_middle_name,clean_last_name
184,JOSEPH,STEPHEN,ALPERT


In [67]:
# add back to main data set
multi_apps_ids = np.concatenate([
        people_match8.raw_uuid.dropna().unique(), people_match8.raw_uuid_1.dropna().unique(), 
        people_match8.raw_uuid_2.dropna().unique(), people_match8.raw_uuid_3.dropna().unique()], 
                        axis=0)

In [68]:
one_app = get_nonmatched(full_matches2, multi_apps_ids, 'raw_uuid')

(4036, 78)


In [69]:
full_apps = pd.concat([one_app, people_match8], axis=0).sort_values(NAME_COLS+UUID_COLS)
print full_apps.shape

(4155, 81)


In [70]:
dups1 = full_apps[full_apps.duplicated(['clean_last_name', 'clean_first_initial', 'clean_middle_initial', 'medical_school'], keep='first')].sort_values('clean_last_name')
dups2 = full_apps[full_apps.duplicated(['clean_last_name', 'clean_first_initial', 'clean_middle_initial', 'medical_school'], keep='last')].sort_values('clean_last_name')
dups2 = dups2[['clean_last_name', 'clean_first_name', 'clean_first_initial', 'medical_school', 'raw_uuid', 'raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3', 'application_year', 'application_year_1']]

In [71]:
dups3 = dups2.rename(columns={'raw_uuid': 'raw_uuid_4', 'raw_uuid_1': 'raw_uuid_5', 'raw_uuid_2': 'raw_uuid_6',
                             'raw_uuid_3': 'raw_uuid_7', 'application_year': 'application_year_2', 
                              'application_year_1': 'application_year_3'})

In [72]:
dups4 = pd.merge(left=dups1, right=dups3, on=['clean_last_name', 'clean_first_initial', 'medical_school'], how='inner')
dups4.loc[:, ['raw_uuid', 'raw_uuid_4', 'raw_uuid_1', 'raw_uuid_5', 'raw_uuid_2', 'raw_uuid_6',
                             'raw_uuid_3', 'raw_uuid_7', 'application_year', 'application_year_2', 
                              'application_year_1', 'application_year_3']]
dups4.loc[pd.isnull(dups4.application_year_1), 'application_year_1'] =  dups4.loc[
    pd.isnull(dups4.application_year_1), 'application_year_3'] 
dups4.loc[pd.isnull(dups4.application_year_1), 'application_year_1'] =  dups4.loc[
    pd.isnull(dups4.application_year_1), 'application_year_2']
dup_years = dups4.application_year_1==dups4.application_year_2
dups4.loc[dup_years, 'application_year_2'] = np.nan


In [73]:
dups5 = dups4.join(dups4[['raw_uuid', 'raw_uuid_4', 'raw_uuid_1', 'raw_uuid_5', 'raw_uuid_2', 'raw_uuid_6',
                             'raw_uuid_3', 'raw_uuid_7']].apply(get_unique_vals, axis=1))

In [74]:
dups6 = dups5.rename(columns=dict(zip(range(6), ['raw_uuid', 'raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3', 'raw_uuid_4',
                             'raw_uuid_5'])))
sorted(dups6.columns)
dups6 = dups5[['raw_uuid', 'raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3', 'raw_uuid_4',
                'raw_uuid_5', 'clean_last_name', 'clean_first_initial', 'medical_school', 
                   'application_year', 'application_year_2', 
                              'application_year_1',  'application_year_3']]

In [75]:
not_dups = full_apps[
    ~full_apps.duplicated(
        ['clean_last_name', 'clean_first_initial', 'clean_middle_initial', 'medical_school'], keep=False)]

In [76]:
full_apps1 = pd.concat([not_dups, dups6], axis=0).reset_index(drop=True)

In [77]:
print full_matches.shape

(4274, 87)


In [78]:
all_last_names = set(all_app7.clean_last_name.values)
merged_last_names = set(full_apps.clean_last_name.values)
diff_names = all_last_names - merged_last_names
print diff_names

set(['HUNT', 'COLLIN', 'BULL', 'LIST', 'GARFIN', 'FINKLESTEIN', 'HERSH', 'ROBINS'])


In [79]:
set(full_apps1.columns) - set(all_app7a.columns)

{'application_year_1',
 'application_year_2',
 'application_year_3',
 'fuzzy_merge_col',
 'index',
 'is_match',
 'last_name_counts',
 'raw_uuid_1',
 'raw_uuid_2',
 'raw_uuid_3',
 'raw_uuid_4',
 'raw_uuid_5'}

In [80]:
# need to add back in people who were dropped after renaming columns to match full_apps1
missing_ppl = all_app7a.loc[(
        all_app7a.clean_last_name.isin(diff_names)) & (~pd.isnull(all_app7a.clean_first_name))]

In [81]:
full_apps1A = pd.concat([
        full_apps1, missing_ppl], axis=0).reset_index(drop=True).drop(['index', 'is_match', 'last_name_counts', 
                                                           'fuzzy_merge_col'], axis=1)

In [82]:
full_apps1A['medical_school'] = full_apps1A.medical_school.apply(clean_med_school)

In [83]:
full_apps1A.index.name = PERSON_ID
print PERSON_ID in full_apps1A.columns
full_apps2 = full_apps1A.reset_index(drop=False).sort_values(NAME_COLS+['medical_school']).drop_duplicates(NAME_COLS)
# .rename(columns={'index': PERSON_ID})
# sorted(full_apps2.columns)

False


In [84]:
full_apps2.loc[full_apps2.medical_school=='VIRGINIA', ['medical_school', 'original_medical_school']]

,medical_school,original_medical_school


In [85]:
possible_drops = full_apps2.loc[(pd.isnull(full_apps2.clean_first_name)&pd.isnull(full_apps2.clean_middle_name))].clean_last_name.values 

# full_apps2.loc[full_apps2.duplicated(NAME_COLS, keep=False), NAME_COLS+['medical_school']].clean_last_name.values

In [86]:
to_drop = ['BURNBAUM', 'DEPPERMAN', 'DEPPERMANN', 'LIST', 'STEWART', 'STEVENS', 'PERPICH', 'BUCKLEY', 'COLLIN']

In [87]:
lname= 'LIST'
not_dups.loc[not_dups.clean_last_name==lname, NAME_COLS+['medical_school']]
one_app.loc[one_app.clean_last_name==lname, NAME_COLS+['medical_school']]
full_matches2.loc[full_matches2.clean_last_name==lname, NAME_COLS+['medical_school']]
full_apps1.loc[full_apps1.clean_last_name==lname, NAME_COLS+['medical_school']]
full_apps2.loc[full_apps2.clean_last_name.isin(to_drop), NAME_COLS+['medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,medical_school
601,DAVID,ALEC,STEVENS,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE
2004,JOSEPH,GEORGE,PERPICH,UNIVERSITY OF MINNESOTA MEDICAL SCHOOL - DULUTH
2565,MICHAEL,M,STEWART,NaN
2629,MITCHELL,DAVID,BURNBAUM,MEDICAL COLLEGE OF VIRGINIA
2644,MORTIMER,NaN,BUCKLEY,NaN
4150,NOEL,DAVID,LIST,SUNY DOWNSTATE MEDICAL CENTER COLLEGE OF MEDICINE
4149,ROBERT,CHAPLIN,COLLIN,JOAN & SANFORD I. WEILL MEDICAL COLLEGE CORNEL...
3532,SCOTT,NaN,STEWART,CASE WESTERN RESERVE UNIVERSITY SCHOOL OF MEDI...
3664,STEPHEN,ROLFE,DEPPERMAN,UNIVERSITY OF PENNSYLVANIA SCHOOL OF MEDICINE
3665,STEPHEN,ROLFE,DEPPERMANN,UNIVERSITY OF PENNSYLVANIA SCHOOL OF MEDICINE


In [88]:

full_apps2['to_drop'] = 0
full_apps2.loc[:, 'to_drop'] = full_apps2[['clean_first_name', 'clean_last_name']].apply(lambda x: pd.isnull(x[0])
                                                                                           and x[1] in(to_drop), axis=1)

In [89]:
c = Counter(full_apps2.clean_last_name)

In [90]:
full_apps2.loc[full_apps2.clean_last_name=='DEPPERMAN', 'to_drop'] = 1 

In [91]:
full_apps2['counts'] = full_apps2.clean_last_name.apply(lambda x: c[x])

In [92]:
possible_drops = full_apps2.loc[
    (pd.isnull(full_apps2.clean_first_name)&pd.isnull(full_apps2.clean_middle_name) & (full_apps2.counts > 1)), 'person_uuid']

In [93]:
full_apps2.loc[full_apps2.person_uuid.isin(possible_drops), 'to_drop'] = 1 

In [94]:
full_apps2.loc[full_apps2.clean_last_name=='BURNBAUM', NAME_COLS+['application_year']]

,clean_first_name,clean_middle_name,clean_last_name,application_year
2629,MITCHELL,DAVID,BURNBAUM,1972.0
4096,NaN,DAVID,BURNBAUM,1972.0


In [95]:
full_apps2.loc[full_apps2.to_drop==1,  NAME_COLS+['application_year']]

,clean_first_name,clean_middle_name,clean_last_name,application_year
3664,STEPHEN,ROLFE,DEPPERMAN,1970.0
4096,NaN,DAVID,BURNBAUM,1972.0
4110,NaN,M,STEWART,1965.0
4126,NaN,NaN,ALMOND,1962.0
4127,NaN,NaN,ANDERSON,1961.0
4129,NaN,NaN,BORNSTEIN,1960.0
4130,NaN,NaN,BROWN,1961.0
4132,NaN,NaN,GAY,1963.0
4133,NaN,NaN,GOLDBERG,1963.0
4134,NaN,NaN,GROSSMAN,1960.0


In [96]:
full_apps3 = full_apps2.loc[full_apps2.to_drop==0]

In [97]:
# write to csv
full_apps3.to_csv(os.path.join(APP_DATA_DIR, 'index_cards_deduped_fuzzy.csv'), index=False)
full_apps3.to_pickle(os.path.join(PICKLE_DIR, 'index_cards_deduped_fuzzy.p'))


In [98]:
# also write out original raw index card (pre merge data set)
all_app7.to_csv(os.path.join(APP_DATA_DIR, 'index_cards_raw.csv'), index=False)
all_app7.to_pickle(os.path.join(PICKLE_DIR, 'index_cards_raw.p'))

In [99]:
last_name='HERSH'

full_apps2.loc[full_apps2.clean_last_name==last_name, NAME_COLS+[PERSON_ID, 'medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,person_uuid,medical_school
4148,EVAN,M,HERSH,4148,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...


In [100]:
# do some sanity checks on the data
all_last_names = set(all_app7.clean_last_name.values)
merged_last_names = set(full_apps2.clean_last_name.values)
diff_names = all_last_names - merged_last_names
print diff_names

set(['FINKLESTEIN', 'HUNT', 'ROBINS'])


In [101]:
print UUID_COLS, APPLICATION_YEAR_COLS

['raw_uuid', 'raw_uuid_1', 'raw_uuid_2', 'raw_uuid_3'] ['application_year', 'application_year_1']


In [102]:
missing_ppl = all_app7.loc[all_app7.clean_last_name.isin(diff_names), NAME_COLS+['raw_uuid', 'application_year']]

In [103]:
full_apps.loc[full_apps.raw_uuid.isin(missing_ppl.raw_uuid) | full_apps.raw_uuid_1.isin(missing_ppl.raw_uuid), NAME_COLS]

,clean_first_name,clean_middle_name,clean_last_name
1090,JAMES,NaN,FINKELSTEIN
1804,ROBERT,D,HUNTER
3207,SANDER,NaN,ROBINSON


In [104]:
all_app7.loc[all_app7.clean_last_name=='CHISARI']

,address,age,application_date,application_year,associate_program_entered,bob,ca,cc,citizenship,city,...,state,teaching,undergrad_year_grad,undergraduate_school,withdrawal,year_accepted,zip_code,clean_first_initial,clean_middle_initial,fuzzy_merge_col
4311,445 E. 69th St.,NaN,1968-01-01,1968.0,NaN,0.0,1.0,1.0,NaN,New York,...,New York,1.0,NaN,NaN,-9.0,1970,10021.0,F,V,FRANCIS VINCENT CHISARI
233,445 East 69th Street,NaN,1968-01-01,1968.0,NaN,0.0,1.0,0.0,NaN,New York,...,New York,1.0,NaN,NaN,-9.0,1970.0,10021,F,V,FRANCIS VINCENT CHISARI
